<div class="demo-info">

(AgileHealth Blood Pressure Dashboard Demo)

This dashboard fetches and displays blood pressure data for a patient.

(DEMO) To edit the dashboard:

1. <a href="#" id="demo-edit-link" target="demo-edit">click here</a> to open the notebook
1. edit the notebook
1. save changes
1. reload this page

</div>

<script type="text/javascript">
var link = document.getElementById("demo-edit-link");
link.href = document.location.href.replace("/voila/render/", "/lab/tree/")
</script>

<style type="text/css">
.demo-info {
    font-style: italic;
    /* hide all the demo info */
    display: none;
}

.added-widget {
    /* add some highlighting for widgets added in the demo */
    font-weight: bold;
    background-color: rgb(220, 255, 220) !important;
}

</style>

In [ ]:
# To add goals to the demo, add `include_goal=True` to the last cell in this notebook

In [ ]:
import os

import requests

s = requests.Session()
s.headers["Authorization"] = f"Bearer {os.environ['CHCS_TOKEN']}"

In [ ]:
# check our credentials:
r = s.get("https://chcs.fly.dev/api/v1/users/profile")
if not r.ok:
    msg = f"Failed to access CHCS endpoint (status={r.status_code}). Logout of JupyterHub, login again, and restart your server to get fresh credentials."
    print(msg)
    raise PermissionError(msg)
print("logged in as:")
user_info = r.json()
user_info

In [ ]:
# select paiient_id
# use user's own patient id if they are a patient,
# otherwise a test id that has data
user_patient = user_info["patient"]
if user_patient:
    # user is a patient, show them their own data
    try:
        int(user_patient)
    except ValueError:
        # it's the "Patient/{id}" string
        patient_id = user_patient
    else:
        # it's the integer
        patient_id = f"Patient/{user_patient}"
else:
    # test patient with bp data
    patient_id = 40001
    # Fernando is 40058

In [ ]:
r = s.get("https://chcs.fly.dev/fhir/r5/Patient")
if not r.ok:
    print(r.text)
    r.raise_for_status()
patients = r.json()["entry"]

if not os.environ.get("VOILA_REQUEST_URL"):
    print("available patients:")
    for patient in patients:
        resource = patient["resource"]
        # print("id:", resource['id'])
        # print("Given name:", *resource['name'][0]['given'])
        # print("Family name:", resource['name'][0]['family'])
        email = None
        for item in resource["telecom"]:
            if item["system"] == "email":
                email = item["value"]
                # print("email:", item["value"])
        if email:
            print(resource["id"], email, f"{resource['name'][0]['family']},", " ".join(resource['name'][0]['given']))

In [ ]:
# fetch observations
r = s.get(
    "https://chcs.fly.dev/fhir/r5/Observation",
    params={
        # Study id
        "_has:Group:member:_id": 30004,
        # Patient id
        "patient": patient_id,
        # only want blood pressure
        "code": "https://w3id.org/openmhealth|omh:blood-pressure:4.0",
    },
)
r.raise_for_status()
observations = r.json()["entry"]

if not os.environ.get("VOILA_REQUEST_URL"):
    print(len(observations))
    print("Found data for these patients:")
    seen = set()
    for entry in observations:
        subj = entry["resource"]["subject"]["reference"]
        if subj not in seen:
            seen.add(subj)
            print(subj)
    len(observations)
if not len(observations):
    raise ValueError("Found no observations!")

In [ ]:
# load the blood pressure Observations into pandas

import base64
import json

import pandas as pd
from commonhealth_cloud_storage_client import ResourceHolder
from jupyter_health.utils import tidy_record

BLOOD_PRESSURE = "omh:blood-pressure:4.0"

records = []

for observation in observations:
    resource = observation["resource"]
    code = resource["code"]["coding"][0]["code"]
    json_record = json.loads(base64.b64decode(resource["valueAttachment"]["data"]))
    if "body" in json_record:
        # old format
        continue
    holder = ResourceHolder({"header": {}, "body": json_record}, code)
    try:
        record = tidy_record(holder)
    except Exception as e:
        # old schema
        print(repr(e), resource["id"])
        continue
    record["subject"] = resource["subject"]["reference"]
    records.append(record)

df = patient_df = pd.DataFrame.from_dict(records)
if not len(df):
    raise ValueError(
        f"Found no bp records for {patient_id} in {len(observations)} observations"
    )
if not os.environ.get("VOILA_REQUEST_URL"):
    print(len(df))

In [ ]:
# now start the plotting

import os
from datetime import timedelta
from enum import Enum
from functools import partial
from itertools import chain

import altair as alt
import pandas as pd
from IPython.display import display

pd.options.mode.chained_assignment = None


class Goal(Enum):
    """Enum for met/unmet

    These strings will be used for the legend.
    """

    met = "under"
    unmet = "over"


class Category(Enum):
    normal = "normal"
    elevated = "elevated"
    hypertension = "hypertension"


def classify_bp(row):
    """Classify blood pressure"""
    # https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings
    # note from : We can decide to have just normal, elevated and hypertension to begin with
    if (
        row.diastolic_blood_pressure_value < 80
        and row.systolic_blood_pressure_value < 120
    ):
        return Category.normal.value
    elif (
        row.diastolic_blood_pressure_value < 80
        and 120 <= row.systolic_blood_pressure_value < 130
    ):
        return Category.elevated.value
    else:
        return Category.hypertension.value


def bp_goal(patient_df, goal="140/90"):
    """True/False for blood pressure met goal"""
    sys_goal, dia_goal = (int(s) for s in goal.split("/"))
    if (patient_df.systolic_blood_pressure_value <= sys_goal) & (
        patient_df.diastolic_blood_pressure_value <= dia_goal
    ):
        return Goal.met.value
    else:
        return Goal.unmet.value


red_yellow_blue = [
    "#4a74b4",
    "#faf8c1",
    "#d4322c",
]
red_blue = [red_yellow_blue[0], red_yellow_blue[-1]]


def bp_over_time(bp, color_scale="category"):
    """Plot blood pressure over time"""
    # https://vega.github.io/vega/docs/schemes/#redyellowblue
    if color_scale == "category":
        domain = [
            Category.normal.value,
            Category.elevated.value,
            Category.hypertension.value,
        ]
        color = alt.Color(
            "category:O",
            scale=alt.Scale(
                domain=domain,
                range=red_yellow_blue,
            ),
        )
        shape = alt.Shape(
            "category:O",
            scale=alt.Scale(
                domain=domain,
            ),
        )
    elif color_scale == "goal":
        domain = [Goal.met.value, Goal.unmet.value]
        color = alt.Color(
            "goal:O",
            scale=alt.Scale(domain=domain, range=red_blue),
        )
        shape = alt.Shape(
            "goal:O",
            scale=alt.Scale(domain=domain),
        )

    # heuristic for x-ticks
    end_time = bp.effective_time_frame_date_time.max()
    end_date = end_time.date()
    start_date = bp.effective_time_frame_date_time.min().date()
    time_frame_days = (end_date - start_date).total_seconds() / (3600 * 24)
    axis_args = {"format": "%Y-%m-%d"}
    if time_frame_days < 7:
        # minimum of one week
        start_date = end_date - timedelta(days=7)
    if time_frame_days < 14:
        # at least a week
        axis_args["tickCount"] = dict(interval="day", step=1)
    if 14 <= time_frame_days < 30:
        # expand less than a month to 1 month
        start_date = end_date - timedelta(days=30)
    if time_frame_days > 90:
        # at least a few months, label with year-month
        axis_args["format"] = "%Y-%m"

    x = alt.X(
        "effective_time_frame_date_time_local",
        title="date",
        axis=alt.Axis(
            labelAngle=30,
            **axis_args,
        ),
        scale=alt.Scale(
            domain=[
                pd.to_datetime(start_date),
                pd.to_datetime(end_date + timedelta(days=1)),
            ]
        ),
    )

    charts = [
        [
            alt.Chart(bp, title="blood pressure")
            .mark_line(color="#333")
            .encode(
                x=x,
                y=alt.Y(f"{which}_blood_pressure_value", title="mmHg"),
            ),
            alt.Chart(bp, title="blood pressure")
            .mark_point(filled=True)
            .encode(
                x=x,
                y=alt.Y(f"{which}_blood_pressure_value", title="mmHg"),
                color=color,
                shape=shape,
                tooltip=[
                    alt.Tooltip("effective_time_frame_date_time_local", title="date"),
                    alt.Tooltip("systolic_blood_pressure_value", title="Systolic"),
                    alt.Tooltip("diastolic_blood_pressure_value", title="Diastolic"),
                    alt.Tooltip("category"),
                ],
            ),
        ]
        for which in ("systolic", "diastolic")
    ]
    return alt.layer(*chain(*charts))


def bp_by_tod(bp):
    """Plot blood pressure by time of day"""
    tod_tooltip = [
        alt.Tooltip(
            "mean(diastolic_blood_pressure_value):Q",
            title="avg diastolic",
            format=".0f",
        ),
        alt.Tooltip(
            "mean(systolic_blood_pressure_value):Q", title="avg systolic", format=".0f"
        ),
        alt.Tooltip("count(diastolic_blood_pressure_value)", title="measurements"),
        alt.Tooltip("hours(effective_time_frame_date_time_local)", title="hour"),
    ]
    charts = [
        alt.Chart(bp, title="by time of day")
        .mark_line(point=True)
        .encode(
            x=alt.X(
                "hours(effective_time_frame_date_time_local)",
                title="time of day",
                scale=alt.Scale(domain=[0, 24]),
            ),
            y=alt.Y(
                f"mean({which}_blood_pressure_value):Q",
                title=which,
                axis=alt.Axis(title=""),
            ),
            tooltip=tod_tooltip,
        )
        for which in ("systolic", "diastolic")
    ]
    return alt.layer(*charts)


def plot_patient_blood_pressure(patient_df, goal="140/90", color_scale="category"):
    """plot blood pressure, given a patient data frame, as returned by get_patient_data"""
    bp = patient_df.loc[patient_df.resource_type == BLOOD_PRESSURE]
    bp["category"] = bp.apply(classify_bp, axis=1)
    bp["goal"] = bp.apply(partial(bp_goal, goal=goal), axis=1)
    return (
        (bp_over_time(bp, color_scale=color_scale) | bp_by_tod(bp))
        .resolve_scale(y="shared", color="independent", shape="independent")
        .configure_point(size=100)
        .interactive()
    )


if not os.environ.get("VOILA_REQUEST_URL"):
    # show last 30 days of data
    end_date = df.effective_time_frame_date_time.max()
    display(
        plot_patient_blood_pressure(
            df[df.effective_time_frame_date_time >= (end_date - timedelta(days=30))],
            color_scale="goal",
        )
    )

In [ ]:
import ipywidgets as W
import pandas as pd
from IPython.display import HTML


def bp_category_style(row):
    """highlight rows by bp category"""
    category = Category(row.category)
    if category == Category.hypertension:
        color = "#fdd"
    elif category == Category.elevated:
        color = "#ffc"
    else:
        color = None

    return [f"background-color:{color}" if color else None] * len(row)


def bp_goal_style(row):
    """highlight rows by bp category"""
    goal = Goal(row.goal)
    if goal == Goal.unmet:
        color = "#fdd"
    else:
        color = None

    return [f"background-color:{color}" if color else None] * len(row)


def bp_goal_fraction(patient_df, goal="140/90"):
    bp = patient_df.loc[patient_df.resource_type == BLOOD_PRESSURE]
    sys_goal, dia_goal = (int(s) for s in goal.split("/"))
    met_goal = (bp.systolic_blood_pressure_value <= sys_goal) & (
        bp.diastolic_blood_pressure_value <= dia_goal
    )
    return met_goal.sum() / len(bp)


def bp_table_info(patient_df, goal="140/90", color_scale="category"):
    """Display tabular info"""
    bp = patient_df.loc[
        patient_df.resource_type == BLOOD_PRESSURE,
        [
            "effective_time_frame_date_time_local",
            "systolic_blood_pressure_value",
            "diastolic_blood_pressure_value",
        ],
    ]
    bp["category"] = bp.apply(classify_bp, axis=1)
    bp["goal"] = bp["goal"] = bp.apply(partial(bp_goal, goal=goal), axis=1)
    # relabel columns
    bp.columns = ["date", "systolic", "diastolic", "category", "goal"]
    bp["time"] = bp.date.dt.time
    bp["date"] = bp.date.dt.date
    bp = bp.astype({"systolic": int, "diastolic": int})

    label_style = {"font_weight": "bold", "font_size": "150%"}

    table = W.Output()
    with table:
        styled = (
            bp.style.hide()
            .hide(["category", "goal"], axis="columns")
            .format({"time": "{:%H:%M}"})
        )
        if color_scale == "goal":
            styled = styled.apply(bp_goal_style, axis=1)
        else:
            styled = styled.apply(bp_category_style, axis=1)
        display(HTML(styled.to_html(index=False)))

    summary = W.Output()
    min_idx = bp.systolic.idxmin()
    max_idx = bp.systolic.idxmax()
    summary_table = pd.DataFrame(
        {
            "systolic": [
                bp.systolic.min(),
                bp.systolic.max(),
                bp.systolic.mean(),
            ],
            "diastolic": [
                bp.diastolic.min(),
                bp.diastolic.max(),
                bp.diastolic.mean(),
            ],
            "date": [
                bp.loc[min_idx].date,
                bp.loc[max_idx].date,
                "-",
            ],
        }
    )
    summary_table = summary_table.astype({"systolic": int, "diastolic": int})
    summary_table.index = pd.Index(["min", "max", "avg"])
    with summary:
        display(HTML(summary_table.to_html()))

    # calculate goal fraction
    at_goal_fraction = bp_goal_fraction(patient_df, goal)
    overview = W.Output()
    with overview:
        display(
            HTML(f"<div style='font-size: 250%'>at goal: {at_goal_fraction:.0%}</div>")
        )

    box_layout = {
        "border_left": "1px solid #aaa",
        "padding": "8px",
        "margin": "8px",
    }
    right_box = [
        W.Label(value="summary", style=label_style),
        summary,
    ]
    if color_scale == "goal":
        right_box.extend(
            [
                W.Label(value="overview", style=label_style),
                overview,
            ]
        )
    layout = W.HBox(
        [
            W.VBox(
                [W.Label(value="Measurements", style=label_style), table],
                layout=box_layout,
            ),
            W.VBox(
                right_box,
                layout=box_layout,
            ),
        ]
    )
    layout.layout.justify_content = "flex-start"
    return layout


if not os.environ.get("VOILA_REQUEST_URL"):
    # preview while run interactively (not voila)
    display(bp_table_info(df[-100:], color_scale="goal"))

In [ ]:
import time


def plot_patient(
    view=30,
    comorbidity=False,
    color="goal",
):
    if comorbidity:
        goal = "130/80"
    else:
        goal = "140/90"
    df = patient_df

    last_day = df.effective_time_frame_date_time.dt.date.max()
    start_date = last_day - timedelta(days=view)
    df = df[df.effective_time_frame_date_time.dt.date >= start_date]
    # for demo: current user is too healthy
    df = df.copy()
    df["systolic_blood_pressure_value"] = (
        df["systolic_blood_pressure_value"] * 1.1
    ).astype(int)
    df["diastolic_blood_pressure_value"] = (
        df["diastolic_blood_pressure_value"]
    ).astype(int)
    display(plot_patient_blood_pressure(df, goal=goal, color_scale=color))
    display(bp_table_info(df, goal=goal, color_scale=color))
    # utterly bizarre: voilá hangs if this returns too quickly
    time.sleep(0.5)


if not os.environ.get("VOILA_REQUEST_URL"):
    plot_patient()

<div class="demo-info">
    
Everything above here is setup of the demo itself, not part of the demo; this would all be hidden and managed by EHR inputs.
Below here is the actual demo.

</div>

## AgileHealth Dashboard

In [ ]:
# create the interactive chart


def create_dashboard(include_goal=False):
    """
    Create the dashboard

    only one input: include_goal
    if True, include UCSF goal inputs ('after' view)
    if False, only include American Heart Association category view
    """

    interact_args = {}
    if include_goal:
        comorbidity_widget = W.Checkbox(
            description="Any comorbidities: ASCVD > 10%, Diabetes Mellitus, CKD (EGFR 20-59), Heart Failure"
        )
        comorbidity_widget.layout.width = "100%"
        comorbidity_widget.add_class("added-widget")
        color_widget = W.Dropdown(
            value="goal", options={"UCSF Goal": "goal", "AHA Category": "category"}
        )
        color_widget.add_class("added-widget")
        interact_args["comorbidity"] = comorbidity_widget
        interact_args["color"] = color_widget
    else:
        interact_args["comorbidity"] = W.fixed(False)
        interact_args["color"] = W.fixed("category")

    dashboard = W.interactive(
        plot_patient,
        view={
            "Week": 7,
            "Month": 30,
            "Year": 365,
        },
        **interact_args,
    )
    # give it some border to set it off from the demo setup
    dashboard.layout.border = "4px solid #cff"
    dashboard.layout.padding = "16px 30px"

    # rerender on change of widgets outside the interact
    # patient_widget.observe(lambda change: dashboard.update(), names="value")
    # partner_widget.observe(lambda change: dashboard.update(), names="value")
    return dashboard


# DEMO: uncomment include_goal=True to add UCSF goals to visualization,
# save, and refresh dashboard


create_dashboard(
    include_goal=True,
)